In [16]:
import joblib

Leitura e tratamento da tabela prospects

In [17]:
import json
import pandas as pd

with open("prospects.json", "r", encoding="utf-8") as f:
    prospects_data = json.load(f)

prospects_list = []

for vaga_id, vaga_info in prospects_data.items():
    titulo = vaga_info.get("titulo", "")
    for prospect in vaga_info.get("prospects", []):
        candidato = {
            "vaga_id": vaga_id,
            "titulo_vaga": titulo,
            "codigo_candidato": prospect.get("codigo"),
            "nome": prospect.get("nome"),
            "situacao": prospect.get("situacao_candidado"),
            "data_candidatura": prospect.get("data_candidatura"),
            "recrutador": prospect.get("recrutador"),
            "contratado": 1 if prospect.get("situacao_candidado") == "Contratado pela Decision" else 0
        }
        prospects_list.append(candidato)

df_prospects = pd.DataFrame(prospects_list)

df_prospects.head()


,vaga_id,titulo_vaga,codigo_candidato,nome,situacao,data_candidatura,recrutador,contratado
0,4530,CONSULTOR CONTROL M,25632,José Vieira,Encaminhado ao Requisitante,25-03-2021,Ana Lívia Moreira,0
1,4530,CONSULTOR CONTROL M,25529,Srta. Isabela Cavalcante,Encaminhado ao Requisitante,22-03-2021,Ana Lívia Moreira,0
2,4531,2021-2607395-PeopleSoft Application Engine-Dom...,25364,Sra. Yasmin Fernandes,Contratado pela Decision,17-03-2021,Juliana Cassiano,1
3,4531,2021-2607395-PeopleSoft Application Engine-Dom...,25360,Alexia Barbosa,Encaminhado ao Requisitante,17-03-2021,Juliana Cassiano,0
4,4533,2021-2605708-Microfocus Application Life Cycle...,26338,Arthur Almeida,Contratado pela Decision,29-04-2021,Stella Vieira,1


Leitura do applicants e união com o prospects

In [18]:
with open("applicants.json", "r", encoding="utf-8") as f:
    applicants_data = json.load(f)

codigos_relevantes = set(df_prospects["codigo_candidato"].astype(str).unique())

applicants_list = []
for codigo, dados in applicants_data.items():
    if codigo in codigos_relevantes:
        base = {"codigo_candidato": codigo}
        for secao in dados:
            if isinstance(dados[secao], dict):
                for k, v in dados[secao].items():
                    base[f"{secao}_{k}"] = v
            else:
                base[secao] = dados[secao]
        applicants_list.append(base)

df_applicants = pd.DataFrame(applicants_list)

df_merged = df_prospects.merge(df_applicants, on="codigo_candidato", how="left")

df_merged.head()


,vaga_id,titulo_vaga,codigo_candidato,nome,situacao,data_candidatura,recrutador,contratado,infos_basicas_telefone_recado,infos_basicas_telefone,...,cargo_atual_id_ibrati,cargo_atual_email_corporativo,cargo_atual_cargo_atual,cargo_atual_projeto_atual,cargo_atual_cliente,cargo_atual_unidade,cargo_atual_data_admissao,cargo_atual_data_ultima_promocao,cargo_atual_nome_superior_imediato,cargo_atual_email_superior_imediato
0,4530,CONSULTOR CONTROL M,25632,José Vieira,Encaminhado ao Requisitante,25-03-2021,Ana Lívia Moreira,0,,(21) 93485-6494,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4530,CONSULTOR CONTROL M,25529,Srta. Isabela Cavalcante,Encaminhado ao Requisitante,22-03-2021,Ana Lívia Moreira,0,,(21) 94261-6276,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4531,2021-2607395-PeopleSoft Application Engine-Dom...,25364,Sra. Yasmin Fernandes,Contratado pela Decision,17-03-2021,Juliana Cassiano,1,,(21) 96332-6963,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4531,2021-2607395-PeopleSoft Application Engine-Dom...,25360,Alexia Barbosa,Encaminhado ao Requisitante,17-03-2021,Juliana Cassiano,0,,(11) 97744-8180,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4533,2021-2605708-Microfocus Application Life Cycle...,26338,Arthur Almeida,Contratado pela Decision,29-04-2021,Stella Vieira,1,,(31) 92702-5791,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Leitura e união com  arquivo de vagas

In [19]:
with open("vagas.json", "r", encoding="utf-8") as f:
    vagas_data = json.load(f)

vagas_list = []
for vaga_id, dados in vagas_data.items():
    base = {"vaga_id": vaga_id}
    for secao in dados:
        if isinstance(dados[secao], dict):
            for k, v in dados[secao].items():
                base[f"{secao}_{k}"] = v
    vagas_list.append(base)

df_vagas = pd.DataFrame(vagas_list)

df_final = df_merged.merge(df_vagas, on="vaga_id", how="left")

print("Colunas disponíveis:", df_final.columns.tolist())

df_final[["codigo_candidato", "vaga_id", "titulo_vaga", "perfil_vaga_nivel profissional", "perfil_vaga_nivel_ingles", "perfil_vaga_areas_atuacao"]].head()


Colunas disponíveis: ['vaga_id', 'titulo_vaga', 'codigo_candidato', 'nome', 'situacao', 'data_candidatura', 'recrutador', 'contratado', 'infos_basicas_telefone_recado', 'infos_basicas_telefone', 'infos_basicas_objetivo_profissional', 'infos_basicas_data_criacao', 'infos_basicas_inserido_por', 'infos_basicas_email', 'infos_basicas_local', 'infos_basicas_sabendo_de_nos_por', 'infos_basicas_data_atualizacao', 'infos_basicas_codigo_profissional', 'infos_basicas_nome', 'informacoes_pessoais_data_aceite', 'informacoes_pessoais_nome', 'informacoes_pessoais_cpf', 'informacoes_pessoais_fonte_indicacao', 'informacoes_pessoais_email', 'informacoes_pessoais_email_secundario', 'informacoes_pessoais_data_nascimento', 'informacoes_pessoais_telefone_celular', 'informacoes_pessoais_telefone_recado', 'informacoes_pessoais_sexo', 'informacoes_pessoais_estado_civil', 'informacoes_pessoais_pcd', 'informacoes_pessoais_endereco', 'informacoes_pessoais_skype', 'informacoes_pessoais_url_linkedin', 'informacoes

,codigo_candidato,vaga_id,titulo_vaga,perfil_vaga_nivel profissional,perfil_vaga_nivel_ingles,perfil_vaga_areas_atuacao
0,25632,4530,CONSULTOR CONTROL M,Pleno,Nenhum,TI - Desenvolvimento/Programação-
1,25529,4530,CONSULTOR CONTROL M,Pleno,Nenhum,TI - Desenvolvimento/Programação-
2,25364,4531,2021-2607395-PeopleSoft Application Engine-Dom...,Sênior,Nenhum,Gestão e Alocação de Recursos de TI-
3,25360,4531,2021-2607395-PeopleSoft Application Engine-Dom...,Sênior,Nenhum,Gestão e Alocação de Recursos de TI-
4,26338,4533,2021-2605708-Microfocus Application Life Cycle...,Sênior,Técnico,Gestão e Alocação de Recursos de TI-


Modelagem e treinamento do modelo

In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
import joblib

colunas_modelo = [
    'formacao_e_idiomas_nivel_academico',
    'formacao_e_idiomas_nivel_ingles',
    'formacao_e_idiomas_nivel_espanhol',
    'informacoes_profissionais_area_atuacao',
    'informacoes_profissionais_nivel_profissional',
    'informacoes_profissionais_conhecimentos_tecnicos',
    'informacoes_basicas_tipo_contratacao',
    'perfil_vaga_nivel profissional',
    'perfil_vaga_nivel_ingles',
    'perfil_vaga_areas_atuacao'
]

X = df_final[colunas_modelo]
y = df_final["contratado"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)

categoricas = X.select_dtypes(include='object').columns.tolist()

preprocessador = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown='ignore', sparse_output=False), categoricas)
    ]
)

pipeline = Pipeline([
    ("preprocessador", preprocessador),
    ("modelo", LogisticRegression(
        penalty='l2',
        C=1.0,
        solver='liblinear',
        class_weight='balanced',
        random_state=42,
        max_iter=1000
    ))
])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
print("\n📊 Classification Report no Teste:")
print(classification_report(y_test, y_pred))

joblib.dump(pipeline, "modelo_decision.pkl", compress=9)



📊 Classification Report no Teste:
              precision    recall  f1-score   support

           0       0.98      0.73      0.84     10200
           1       0.13      0.72      0.22       552

    accuracy                           0.73     10752
   macro avg       0.55      0.73      0.53     10752
weighted avg       0.94      0.73      0.81     10752



['modelo_decision.pkl']